In [2]:
import numpy as np # linear algebra
import pandas as pd
import sqlite3

In [4]:

# Step 1: Connect to the IPL database
conn = sqlite3.connect("/Users/margotiamanova/Desktop/DI-Bootcamp/Week10/Day2/DailyChallenge/database.sqlite 2")  # Replace with your actual file path

# Step 2: Load all table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

print("📋 Available Tables:")
print(tables)

# Step 3: Load and print columns for each table
print("\n🔍 Table Structures:")
for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 0", conn)
    print(f"\nTable: {table}")
    print("Columns:", df.columns.tolist())

# Close connection when done
# conn.close()


📋 Available Tables:
['Player', 'Extra_Runs', 'Batsman_Scored', 'Batting_Style', 'Bowling_Style', 'Country', 'Season', 'City', 'Outcome', 'Win_By', 'Wicket_Taken', 'Venue', 'Extra_Type', 'Out_Type', 'Toss_Decision', 'Umpire', 'Team', 'Ball_by_Ball', 'sysdiagrams', 'sqlite_sequence', 'Match', 'Rolee', 'Player_Match']

🔍 Table Structures:

Table: Player
Columns: ['Player_Id', 'Player_Name', 'DOB', 'Batting_hand', 'Bowling_skill', 'Country_Name']

Table: Extra_Runs
Columns: ['Match_Id', 'Over_Id', 'Ball_Id', 'Extra_Type_Id', 'Extra_Runs', 'Innings_No']

Table: Batsman_Scored
Columns: ['Match_Id', 'Over_Id', 'Ball_Id', 'Runs_Scored', 'Innings_No']

Table: Batting_Style
Columns: ['Batting_Id', 'Batting_hand']

Table: Bowling_Style
Columns: ['Bowling_Id', 'Bowling_skill']

Table: Country
Columns: ['Country_Id', 'Country_Name']

Table: Season
Columns: ['Season_Id', 'Man_of_the_Series', 'Orange_Cap', 'Purple_Cap', 'Season_Year']

Table: City
Columns: ['City_Id', 'City_Name', 'Country_id']

Tabl

# Query 1: Select All Columns from Player’s Table

In [8]:
# Execute the query
df_player_match = pd.read_sql_query("SELECT * FROM Ball_by_Ball;", conn)

# Show the first few rows
print(df_player_match.head())

   Match_Id  Over_Id  Ball_Id  Innings_No  Team_Batting  Team_Bowling  \
0    335987        1        1           1             1             2   
1    335987        1        1           2             2             1   
2    335987        1        2           1             1             2   
3    335987        1        2           2             2             1   
4    335987        1        3           1             1             2   

   Striker_Batting_Position  Striker  Non_Striker  Bowler  
0                         1        1            2      14  
1                         1        6            7     106  
2                         2        2            1      14  
3                         2        7            6     106  
4                         2        2            1      14  


# Query 2: Batsman vs Runs

In [ ]:
df_player_match = pd.read_sql_query("SELECT * FROM Player_Match;", conn)

# Show the first few rows
print(df_player_match.head())

In [9]:
query = """
SELECT p.Player_Name, SUM(bs.Runs_Scored) AS Total_Runs
FROM Batsman_Scored bs
JOIN Ball_by_Ball bb 
  ON bs.Match_Id = bb.Match_Id 
 AND bs.Over_Id = bb.Over_Id 
 AND bs.Ball_Id = bb.Ball_Id
JOIN Player p 
  ON bb.Striker = p.Player_Id
GROUP BY p.Player_Name
ORDER BY Total_Runs DESC;

"""

df_runs = pd.read_sql_query(query, conn)
print(df_runs.head(10))  

      Player_Name  Total_Runs
0         V Kohli        8158
1        SK Raina        7588
2       RG Sharma        7263
3       G Gambhir        6793
4      RV Uthappa        6303
5       DA Warner        6235
6        CH Gayle        6105
7        MS Dhoni        6006
8        S Dhawan        5917
9  AB de Villiers        5832


#  Query 3: Fifties and Hundreds

In [14]:
query = """
SELECT 
    p.Player_Name,
    SUM(CASE WHEN innings_runs BETWEEN 50 AND 99 THEN 1 ELSE 0 END) AS Fifties,
    SUM(CASE WHEN innings_runs >= 100 THEN 1 ELSE 0 END) AS Hundreds
FROM (
    SELECT 
        bb.Striker,
        bs.Match_Id,
        SUM(bs.Runs_Scored) AS innings_runs
    FROM Batsman_Scored bs
    JOIN Ball_by_Ball bb 
      ON bs.Match_Id = bb.Match_Id 
     AND bs.Over_Id = bb.Over_Id 
     AND bs.Ball_Id = bb.Ball_Id
    GROUP BY bb.Striker, bs.Match_Id
) AS player_innings
JOIN Player p ON p.Player_Id = player_innings.Striker
GROUP BY p.Player_Name
ORDER BY Hundreds DESC, Fifties DESC;


"""

df_runs = pd.read_sql_query(query, conn)
print(df_runs.head(10))  

      Player_Name  Fifties  Hundreds
0         V Kohli       29        32
1       DA Warner       21        28
2       G Gambhir       32        24
3        CH Gayle       19        24
4       AM Rahane       18        21
5       RG Sharma       36        20
6        SK Raina       44        19
7  AB de Villiers       29        19
8        SE Marsh       15        19
9      RV Uthappa       40        15
